In [ ]:
# Import necessary libraries
import pandas as pd
import ipywidgets as widgets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, clear_output, HTML

In [ ]:
# Load and preprocess data

# Load dataset
df = pd.read_csv("dataset/Dataset.csv")

# Rename columns
df = df.rename(columns={
    'Restaurant Name': 'name',
    'Cuisines': 'cuisines',
    'Average Cost for two': 'cost',
    'Has Online delivery': 'online_order',
    'Has Table booking': 'book_table',
    'Aggregate rating': 'rating',
    'Rating color': 'rating_color',
    'Rating text': 'rating_text'
})

# Drop rows with missing important fields if df is not empty
if not df.empty:
    df = df.dropna(subset=['cuisines', 'rating', 'cost'])
    df = df.reset_index(drop=True)
    # Create a column combining features for potential text analysis
    df['cost_str'] = df['cost'].astype(str)
    df['content'] = (
        df['cuisines'].astype(str) + ' ' +
        df['rating_text'].astype(str) + ' ' +
        df['rating_color'].astype(str) + ' ' +
        df['cost_str']
    )
else:
    # Add placeholder content column if df is empty
    df['content'] = ""

# Initialize TF-IDF Vectorizer (even if empty)
vectorizer = TfidfVectorizer(stop_words='english')

# Fit/transform only if df has data and content
if not df.empty and not df['content'].isnull().all():
    tfidf_matrix = vectorizer.fit_transform(df['content'].fillna(''))
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
else:
    tfidf_matrix = None
    cosine_sim = None
    print("Warning: Dataset is empty or content column could not be generated. Recommendations might not work.")


In [ ]:
# Define the recommendation function
def recommend_restaurants(cuisine, rating_text, price_range, top_n=8):
    
    # Check if df is usable
    if df.empty or 'cost' not in df.columns or 'cuisines' not in df.columns:
        display(HTML("<p style='color:red;'>Error: Dataset is not loaded correctly. Cannot provide recommendations.</p>"))
        return pd.DataFrame()

    # Define Price Range Boundaries
    price_boundaries = {
        1: (0, 200),
        2: (201, 500),
        3: (501, 1000),
        4: (1001, float('inf'))
    }
    min_cost, max_cost = price_boundaries.get(price_range, (0, float('inf')))

    # Initial Filtering
    filtered_df = df.copy()

    # 1. Filter by cuisine (case-insensitive)
    filtered_df['cuisines'] = filtered_df['cuisines'].astype(str)
    filtered_df = filtered_df[filtered_df['cuisines'].str.contains(cuisine, case=False, na=False)]

    # 2. Filter by Price Range
    filtered_df['cost'] = pd.to_numeric(filtered_df['cost'], errors='coerce')
    filtered_df = filtered_df.dropna(subset=['cost'])
    filtered_df = filtered_df[
        (filtered_df['cost'] >= min_cost) & (filtered_df['cost'] <= max_cost)
    ]

    # 3. Filter by experience level, if specified
    if rating_text != 'Any':
        if 'rating_text' in filtered_df.columns:
            filtered_df = filtered_df[filtered_df['rating_text'] == rating_text]
        else:
            display(HTML("<p style='color:red;'>Error: 'rating_text' column missing from data.</p>"))
            return pd.DataFrame()

    # Check if results exist and return
    if not filtered_df.empty:
        print("✅ Found restaurants matching your preferences:")
        # Sort by rating (desc) and cost (asc)
        return filtered_df.sort_values(by=['rating', 'cost'], ascending=[False, True]).head(top_n)[['name', 'cuisines', 'rating', 'cost']]
    else:
        # If no exact matches, inform the user
        price_mapping_labels = {
            1: "Affordable",
            2: "Average",
            3: "Expensive",
            4: "Luxury"
        }
        price_label = price_mapping_labels.get(price_range, f"Range {price_range}")

        display(HTML("<p style='color:orange;'>😔 No restaurants found matching all your specific criteria (Cuisine: '{}', Experience: '{}', Price: {}).</p>"
                     "<p style='color:orange;'>Suggestion: Try broadening your search, for example, by selecting 'Any' experience or adjusting the price range.</p>"
                     .format(cuisine, rating_text, price_label)))
        return pd.DataFrame()


In [ ]:
# Create  widgets

# Use unique cuisines from the loaded df, handle if empty
if not df.empty and 'cuisines' in df.columns:
    cuisine_options = sorted(df['cuisines'].dropna().unique())
else:
    cuisine_options = ['N/A']

cuisine_widget = widgets.Dropdown(
    options=cuisine_options,
    description='Cuisine:',
    style={'description_width': 'initial'}
)

rating_widget = widgets.Dropdown(
    options=['Any', 'Excellent', 'Very Good', 'Good', 'Average', 'Poor'],
    description='Experience:',
    style={'description_width': 'initial'}
)

# Price Slider and Label
price_slider = widgets.IntSlider(
    value=2, # Default to 'Average'
    min=1,
    max=4,
    step=1,
    description='Price Range:',
    style={'description_width': 'initial'},
    continuous_update=False,
    readout=False # Hide numerical readout
)

price_label_widget = widgets.Label()

# Mapping for price number to capitalized text label
price_mapping_labels = {
    1: "Affordable",
    2: "Average",
    3: "Expensive",
    4: "Luxury"
}

# Function to update the label based on slider value
def update_price_label(change):
    price_value = change['new']
    price_label_widget.value = price_mapping_labels.get(price_value, f"Range {price_value}")

# Link slider changes to the label update function
price_slider.observe(update_price_label, names='value')

# Set initial label value
update_price_label({'new': price_slider.value})

# Combine slider and label into an HBox
price_widget_hbox = widgets.HBox([price_slider, price_label_widget])

# Button and Output widget
submit_button = widgets.Button(
    description='Search Restaurants',
    button_style='success',
    tooltip='Click to get recommendations'
)

output = widgets.Output()


In [ ]:
# Define button click handler
def on_button_click(b):
    with output:
        clear_output()
        cuisine = cuisine_widget.value
        rating = rating_widget.value
        price = price_slider.value # Get numerical price value

        # Get capitalized label for the output message
        price_mapping_labels_for_message = {
            1: "Affordable",
            2: "Average",
            3: "Expensive",
            4: "Luxury"
        }
        price_label_for_message = price_mapping_labels_for_message.get(price, f"Range {price}")

        # Print search criteria
        print(f"🔍 Finding restaurants for '{cuisine}', Experience: '{rating}', Price Range: {price_label_for_message}...\n")

        # Get recommendations
        results = recommend_restaurants(cuisine, rating, price)

        # Display results (recommend_restaurants handles 'not found' message)
        if not results.empty:
            # Convert DataFrame to HTML with left alignment
            html_table = results.to_html(index=False)
            styled_html = f"<div style='text-align: left;'>{html_table}</div>"
            display(HTML(styled_html))

# Link button click event to the handler
submit_button.on_click(on_button_click)


In [ ]:
# Display the UI
display(widgets.VBox([
    cuisine_widget,
    rating_widget,
    price_widget_hbox, # Display HBox with slider and label
    submit_button,
    output
]))